In [1]:
import torch
from torch import nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
from torchvision.io import read_image
import torchvision
from torchvision.transforms import v2
from torchvision import transforms
from torchvision.models.convnext import LayerNorm2d
import csv
from PIL import Image
from sklearn.model_selection import train_test_split

torch.manual_seed(0)

In [2]:
torch.cuda.is_available()

True

In [3]:
all_data = np.genfromtxt('/root/ml/data/train.csv', dtype=float, delimiter=',', skip_header=1)
all_dataint = np.genfromtxt('/root/ml/data/train.csv', dtype=int, delimiter=',', skip_header=1)
all_test = np.genfromtxt('/root/ml/data/test.csv', dtype=float, delimiter=',', skip_header=1)
all_testint = np.genfromtxt('/root/ml/data/test.csv', dtype=int, delimiter=',', skip_header=1)

In [4]:
transform = transforms.v2.Compose([
    transforms.v2.RandomApply([
        transforms.v2.RandomOrder([
            transforms.v2.AutoAugment(
                policy=transforms.v2.AutoAugmentPolicy.IMAGENET,
                interpolation=transforms.v2.InterpolationMode.BILINEAR
            ),
            transforms.v2.RandAugment(),
            transforms.v2.TrivialAugmentWide()  
        ])],
        p = 1.0
    ),
    transforms.v2.ToImage(),
    transforms.v2.ToDtype(torch.float32, scale=True),
    transforms.v2.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
class Dataset(torch.utils.data.Dataset):
  '''
  Prepare the dataset for regression
  '''

  def __init__(self, images, X, y, ids, scale_data=True, random=True):
      if not torch.is_tensor(X) and not torch.is_tensor(y):
        # Apply scaling if necessary
        if scale_data:
            X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.ids = ids
        #self.y = torch.log(torch.from_numpy(y))
        self.y = torch.from_numpy(y).cuda()
        self.images = images
      self.random = random
      self.transform = torchvision.transforms.Compose([
        transforms.v2.ToImage(),
        transforms.v2.ToDtype(torch.float32, scale=True),
        transforms.v2.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      ])

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      #img_path = os.path.join(self.path, f'{str(self.ids[i].item())}.jpeg')
      #image = self.transform(read_image(img_path).float())
      image = self.images[self.ids[i]]
      if self.random:
          image = transform(image)
      else:
          image = self.transform(image)
      #image = pretrained_transforms_test(read_image(img_path).half())
      return self.X[i], image, self.y[i]

In [6]:
class TestDataset(torch.utils.data.Dataset):
  '''
  Prepare the dataset for regression
  '''

  def __init__(self, images, X, ids, scale_data=True):
      if not torch.is_tensor(X):
        # Apply scaling if necessary
        if scale_data:
            X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.ids = ids
      self.images = images
      self.transform = transforms.Compose([
        transforms.v2.ToImage(),
        transforms.v2.ToDtype(torch.float32, scale=True),
        transforms.v2.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      ])
  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      #img_path = os.path.join(self.path, f'{str(self.ids[i].item())}.jpeg')
      #image = self.transform(read_image(img_path).float())
      image = self.transform(self.images[self.ids[i]])
      return self.X[i], image, self.ids[i]

In [7]:
ids = all_dataint[:, 0]
X = all_data[:, 1:-6]
Y = all_data[:, -6:]

X_train, X_val, Y_train, Y_val, ids_train, ids_val = train_test_split(X, Y, ids, test_size=0.10, random_state=42)

Y_means = torch.mean(torch.from_numpy(Y_train), 0).cuda()
Y_stds = torch.std(torch.from_numpy(Y_train), 0).cuda()
#X_means = torch.mean(torch.from_numpy(X_train), 0).cuda()
#X_stds = torch.std(torch.from_numpy(X_train), 0).cuda()

ids_test = all_testint[:, 0]
X_test = all_test[:, 1:]

batchsize = 64

#transform = transforms.Compose([
#    transforms.ToTensor(),
#    transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#])

#trainimages = torchvision.datasets.ImageFolder(root="/root/ml/data/train_images/", transform=transform)
#testimages = torchvision.datasets.ImageFolder(root="/root/ml/data/test_images/", transform=transform)

#fulldataset = Dataset('/root/ml/data/train_images/', X, Y, ids)
#traindataset, valdataset = torch.utils.data.random_split(fulldataset, [0.8, 0.2])
trainimages = dict()
for i in ids_train:
    img_path = os.path.join('/root/ml/data/train_images/', f'{i}.jpeg')
    trainimages[i] = Image.open(img_path)
traindatasetaug = Dataset(trainimages, X_train, Y_train, ids_train, random = True)
traindataset = Dataset(trainimages, X_train, Y_train, ids_train, random = False)

valimages = dict()
for i in ids_val:
    img_path = os.path.join('/root/ml/data/train_images/', f'{i}.jpeg')
    valimages[i] = Image.open(img_path)
valdataset = Dataset(valimages, X_val, Y_val, ids_val, random = False)

trainfulldataset = torch.utils.data.ConcatDataset([traindatasetaug, traindataset])
trainloader = torch.utils.data.DataLoader(trainfulldataset, batch_size=batchsize, shuffle=True, num_workers=0)
valloader = torch.utils.data.DataLoader(valdataset, batch_size=batchsize, shuffle=True, num_workers=0)

testimages = dict()
for i in ids_test:
    img_path = os.path.join('/root/ml/data/test_images/', f'{i}.jpeg')
    testimages[i] = Image.open(img_path)
testdataset = TestDataset(testimages, X_test, ids_test)
testloader = torch.utils.data.DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=0)

In [8]:
class ResidualMLPScalars(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self, num_inputs = 163, num_outputs = 1024):
    super().__init__()
    self.num_inputs = num_inputs
    self.num_outputs = num_outputs
    self.layer1 = nn.Sequential(
      nn.Linear(num_inputs, 512),
      nn.LeakyReLU()
    )
    self.layer2 = nn.Sequential(
      nn.Linear(512, 1024),
      nn.LayerNorm(1024),
      nn.LeakyReLU(),
      nn.Dropout(0.2)
    )
    self.layer3 = nn.Sequential(
      nn.Linear(1024, 1024),
      nn.LeakyReLU()
    )
    self.layer4 = nn.Sequential(
      nn.Linear(1024, 512),
      nn.LeakyReLU()
    )
    self.layer5 = nn.Sequential(
      nn.Linear(512, 512),
      nn.LayerNorm(512),
      nn.LeakyReLU(),
      nn.Dropout(0.2)
    )
    self.layer6 = nn.Sequential(
      nn.Linear(512, 256),
      nn.LeakyReLU(),
      nn.Dropout(0.2)
    )
    self.layer7 = nn.Sequential(
      nn.Linear(256, 128),
    )


  def forward(self, X):
    '''
      Forward pass
    '''
    output1 = self.layer1(X)
    output2 = self.layer2(output1)
    output3 = self.layer3(output2) + output2
    output4 = self.layer4(output3)
    output5 = self.layer5(output4) + output4
    output6 = self.layer6(output5)
    output = self.layer7(output6)
    return output

class ResidualMLPFC(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self, num_inputs = 2048, num_outputs = 6):
    super().__init__()
    self.num_inputs = num_inputs
    self.num_outputs = num_outputs
    self.layer1 = nn.Sequential(
      nn.Linear(num_inputs, 512),
      nn.LeakyReLU(inplace=True)
    )
    self.layer2 = nn.Sequential(
      nn.Linear(512, 512),
      nn.LayerNorm(512),
      nn.LeakyReLU(inplace=True),
      nn.Dropout(0.2)
    )
    self.layer3 = nn.Sequential(
      nn.Linear(512, 128),
      nn.LeakyReLU(inplace=True)
    )
    self.layer4 = nn.Sequential(
      nn.Linear(128, 64),
      nn.LayerNorm(64),
      nn.LeakyReLU(inplace=True)
    )
    self.layer5 = nn.Sequential(
      nn.Linear(64, 6),
    )

  def forward(self, X):
    '''
      Forward pass
    '''
    output1 = self.layer1(X)
    output2 = self.layer2(output1) + output1
    output3 = self.layer3(output2)
    output4 = self.layer4(output3)
    output5 = self.layer5(output4)
    return output5 * Y_stds + Y_means

In [9]:
class MyCoolModel(nn.Module):
  '''
    My cool model for regression.
  '''
  def __init__(self):
    super().__init__()
    '''
    self.image_net = torchvision.models.resnet152(weights='DEFAULT')
    #for param in self.image_net.parameters():
    #    param.requires_grad = False
    num_features = self.image_net.fc.in_features
    #print(num_features)
    #self.image_net.fc = nn.Sequential(
    #    nn.Linear(num_features, 1024)
    #)
    self.image_net.fc = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.LayerNorm(512),
        nn.LeakyReLU(),
        nn.Dropout(0.1),
        nn.Linear(512, 256)
    )
    '''
    '''
    self.image_net = torchvision.models.swin_v2_b(weights='DEFAULT')
    num_features = self.image_net.head.in_features
    self.image_net.head = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.LayerNorm(512),
        nn.LeakyReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 256)
    )
    '''
    self.image_net = torchvision.models.convnext_large(weights='DEFAULT')
    n_inputs = None
    for name, child in self.image_net.named_children():
        if name == 'classifier':
            for sub_name, sub_child in child.named_children():
                if sub_name == '2':
                    n_inputs = sub_child.in_features
    n_outputs = 256
    sequential_layers = nn.Sequential(
        LayerNorm2d((1536,), eps=1e-06, elementwise_affine=True),
        nn.Flatten(start_dim=1, end_dim=-1),
        nn.Linear(n_inputs, 2048, bias=True),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(2048, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Linear(2048, n_outputs),
        nn.LogSoftmax(dim=1)
    )
    self.image_net.classifier = sequential_layers
    
    
    self.scalar_net = ResidualMLPScalars()
    self.fc = ResidualMLPFC(256 + 128, 6)
    
  def forward(self, image, scalars):
    resnet_out = self.image_net(image)
    #return resnet_out
    scalar_out = self.scalar_net(scalars)
    results = self.fc(torch.cat((resnet_out, scalar_out), 1))
    #resnet_out = self.downsample_layer(resnet_out)
    #results = self.fc(torch.cat((resnet_out, scalars), 1))
    return results

In [10]:
torch.cuda.empty_cache()

In [11]:
def CalculateR2(guess, labels, means):
    SS_tot = torch.sum((labels - means) ** 2, 0)
    SS_res = torch.sum((guess - labels) ** 2, 0)
    return torch.mean(1 - (SS_res / SS_tot))

In [12]:
'''
model = torchvision.models.convnext_large(weights='DEFAULT')

n_inputs = None
for name, child in model.named_children():
    if name == 'classifier':
        for sub_name, sub_child in child.named_children():
            if sub_name == '2':
                n_inputs = sub_child.in_features
n_outputs = 6

sequential_layers = nn.Sequential(
    LayerNorm2d((1536,), eps=1e-06, elementwise_affine=True),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(n_inputs, 2048, bias=True),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(2048, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048, n_outputs)
)
model.classifier = sequential_layers

model = model.cuda()
'''
model = MyCoolModel().cuda()

In [13]:
#mlp = MLP()
#vgg11 = VGG11().cuda()
#vgg11 = torch.load('./VGG11-RUN-1-EPOCH-2')
#vgg11 = torch.load('./VGG11-MLP-BLANCEDMSE-EPOCH-6')

def BalancedMSELoss(output, target):
    loss = torch.sum(((output - target) / Y_stds)**2)
    return loss

# Define the loss function and optimizer
#loss_function = nn.MSELoss()
loss_function = BalancedMSELoss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
#optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-3)
#optimizer = torch.optim.RMSprop(model.parameters())

In [15]:
# Run the training loop
checkpoints = []
top_score = -100000000000.00
top_epoch = -1
for epoch in range(0, 100): # 5 epochs at maximum
    # Print epoch
    model.train()
    print(f'Starting epoch {epoch + 1}')
    
    # Set current loss value
    current_loss = 0.0
    total_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
    
      # Get and prepare inputs
      inputs, image, targets = data
      inputs, targets = inputs.float(), targets.float()
      targets = targets.reshape((targets.shape[0], 6))
    
      # Zero the gradients
      optimizer.zero_grad()
    
      # Perform forward pass
      #outputs = vgg11(image.cuda(), inputs.cuda())
      outputs = model(image.cuda(), inputs.cuda())
      #outputs = mlp(inputs)
      #print(CalculateR2(outputs, targets, Y_means.cuda()))
    
      # Compute loss
      loss = loss_function(outputs, targets.cuda())
    
      # Perform backward pass
      loss.backward()
      clip = 5
      torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
      # Perform optimization
      optimizer.step()
    
      # Print statistics
      current_loss += loss.item()
      total_loss += loss.item()
      if i % 10 == 9:
          print('Loss after mini-batch %5d: %f' %
                (i + 1, current_loss / (batchsize * 10)))
          current_loss = 0.0
    print (f"Average Train Loss: {total_loss / len(trainloader) / batchsize}")
    model.eval()
    score = 0
    total_loss = 0.0
    for i, data in enumerate(valloader, 0):
        inputs, image, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 6))
        with torch.no_grad():
            outputs = model(image.cuda(), inputs.cuda())
            #score += CalculateR2(torch.exp(outputs), torch.exp(targets.cuda()), Y_means.cuda())
            total_loss += loss_function(outputs, targets.cuda())
            score += CalculateR2(outputs, targets.cuda(), Y_means)
    print(f"SCORE IS: {score / len(valloader)}")
    print(f"Average validation Loss: {total_loss / len(valloader) / batchsize}")
    score = score / len(valloader)
    if score > top_score:
        top_score = score
        top_epoch = epoch
    print(f"TOP SCORE: {top_score} ON EPOCH: {top_epoch}")

    torch.save(model, 'temp_convnext.plt')
    checkpoints.append(torch.load('temp_convnext.plt'))
    
    if (epoch % 10 == 9 or epoch % 10 == 4):
        results = [['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
        for i, data in enumerate(testloader, 0):
            inputs, image, ids = data
            if i % 1000 == 0:
                print(i/6391)
            inputs = inputs.float()
            with torch.no_grad():
                #outputs = torch.exp(vgg11(image.cuda(), inputs.cuda()))
                #outputs = torch.exp(model(image.cuda(), inputs.cuda()))
                outputs = model(image.cuda(), inputs.cuda())
                #outputs = mlp(inputs)
                results.append(ids.tolist() + outputs[0].tolist())
        
        with open(f"results_convnext_full_autoaug_epoch{epoch + 1}.csv","w") as my_csv:
            csvWriter = csv.writer(my_csv, delimiter=',')
            csvWriter.writerows(results)
    torch.cuda.empty_cache()


Starting epoch 1
Loss after mini-batch    10: 4.228468
Loss after mini-batch    20: 4.741458
Loss after mini-batch    30: 4.298169
Loss after mini-batch    40: 4.288399
Loss after mini-batch    50: 4.435717
Loss after mini-batch    60: 3.907275
Loss after mini-batch    70: 3.774861
Loss after mini-batch    80: 3.984678
Loss after mini-batch    90: 4.848957
Loss after mini-batch   100: 3.966998
Loss after mini-batch   110: 4.264110
Loss after mini-batch   120: 4.431697
Loss after mini-batch   130: 4.205693
Loss after mini-batch   140: 4.301761
Loss after mini-batch   150: 3.722799
Loss after mini-batch   160: 4.118874
Loss after mini-batch   170: 4.583405
Loss after mini-batch   180: 4.501888
Loss after mini-batch   190: 4.149472
Loss after mini-batch   200: 4.324503
Loss after mini-batch   210: 4.403068
Loss after mini-batch   220: 4.431045
Loss after mini-batch   230: 4.192840
Loss after mini-batch   240: 4.101342
Loss after mini-batch   250: 4.366985
Loss after mini-batch   260: 4.50

KeyboardInterrupt: 

In [16]:
print(f"BEST SCORE: {top_score} BEST_EPOCH: {top_epoch}")
results = [['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
for i, data in enumerate(testloader, 0):
    inputs, image, ids = data
    if i % 1000 == 0:
        print(i/6391)
    inputs = inputs.float()
    with torch.no_grad():
        #outputs = torch.exp(model(image.cuda(), inputs.cuda()))
        outputs = checkpoints[top_epoch](image.cuda(), inputs.cuda())
        #outputs = mlp(inputs)
        results.append(ids.tolist() + outputs[0].tolist())

with open("results_convnext_learningrate_best.csv","w") as my_csv:
    csvWriter = csv.writer(my_csv, delimiter=',')
    csvWriter.writerows(results)

BEST SCORE: 0.29658438931057546 BEST_EPOCH: 2
0.0
0.15647003598810827
0.31294007197621654
0.4694101079643248
0.6258801439524331
0.7823501799405413
0.9388202159286496
